# Portugese Meals Classification: Optimization

Because of keras abstractions it's really hard to prune pretrained models. I'll stop at just quantization


In [ ]:
import pathlib
import tensorflow as tf
import yaml
import numpy as np

In [ ]:
%cd ..

In [ ]:
config = yaml.safe_load(open('config.yaml', 'r'))

In [ ]:
def uncompress_splits(dir: str):
    X = np.load(dir + 'Xvalues.npz')['arr_0'][:100]
    Y = np.load(dir + 'Yvalues.npz')['arr_0'][:100]

    return X, Y

images, labels = uncompress_splits(config['dataset']['augmented_dir'])

## TFlite

In [ ]:
def representative_dataset():
  for idx in range(100):
    data = images[idx]
    yield [data.astype(np.float32)]

In [ ]:
model = tf.load_model(config['baseline_path'])
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32  
tflite_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path(config['tflite_path'])
tflite_model_file.write_bytes(tflite_model)

## TensorRT

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt


# I'll be using INT8 since I don't have a GPU
conversion_params = trt.TrtConversionParams(
    precision_mode=trt.TrtPrecisionMode.INT8)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=config['saved_model_baseline_path'],
    conversion_params=conversion_params)

converter.convert()

# converter.save(config['tensorrt_path'])
converter.save('./model/tensorrt/')

## ONNX

In [ ]:
%pip install onnxruntime -q
%pip install tf2onnx
%python -m tf2onnx.convert --saved-model tensorflow-model-path --output model.onnx